In [2]:
import pandas as pd 
import numpy as np 
import neo4j

import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
##from pandas_profiling import ProfileReport

In [3]:
data=pd.read_csv('data.tsv',sep='\t')
data.head()
data_1=data.dropna()
data_1.replace("\\N", value = np.nan, inplace = True)
data_1["genres"]=data_1["genres"].replace(np.nan, value = "666")
data_1.infer_objects()

data_1["titleType"]=data_1["titleType"].str.lower()
data_1["primaryTitle"]=data_1["primaryTitle"].str.lower()
data_1["originalTitle"]=data_1["originalTitle"].str.lower()
data_1["genres"]=data_1["genres"].str.lower()
data_1=data_1[data_1["titleType"]=="movie"]


data_1['isAdult']=pd.to_numeric(data_1['isAdult'])
data_1["startYear"]=pd.to_numeric(data_1["startYear"])
data_1["runtimeMinutes"]=pd.to_numeric(data_1["runtimeMinutes"])
data_1["endYear"]=pd.to_numeric(data_1["endYear"])


In [4]:
a=data_1["genres"].unique()
unique_genres = set()

# Iterar sobre cada cadena de géneros en el arreglo
for genres in a:
    # Convertir la cadena en una lista de géneros
    if type(genres) == str:
        genre_list = genres.split(',')
        # Agregar cada género único al conjunto
        unique_genres.update(genre_list)

# Convertir el conjunto de vuelta en una lista
unique_genres_list = list(unique_genres)
print(unique_genres_list)

['sci-fi', 'animation', 'talk-show', 'comedy', 'war', 'horror', 'adventure', 'history', 'thriller', 'music', 'western', 'fantasy', '666', 'film-noir', 'short', 'documentary', 'musical', 'reality-tv', 'sport', 'romance', 'news', 'game-show', 'adult', 'family', 'biography', 'action', 'drama', 'mystery', 'crime']


In [5]:
data_1["tiempo_rank"]=pd.qcut(data_1["runtimeMinutes"], 10,labels = False)

In [6]:
data_1["genres"]=data_1["genres"].astype(str)

In [7]:
data_1['genres'] = [x.strip('()').split(',') for x in data_1['genres']]
data_1.head()


,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres,tiempo_rank
8,tt0000009,movie,miss jerry,miss jerry,0,1894.0,NaN,45.0,[romance],0.0
144,tt0000147,movie,the corbett-fitzsimmons fight,the corbett-fitzsimmons fight,0,1897.0,NaN,100.0,"[documentary, news, sport]",7.0
498,tt0000502,movie,bohemios,bohemios,0,1905.0,NaN,100.0,[666],7.0
570,tt0000574,movie,the story of the kelly gang,the story of the kelly gang,0,1906.0,NaN,70.0,"[action, adventure, biography]",2.0
587,tt0000591,movie,the prodigal son,l'enfant prodigue,0,1907.0,NaN,90.0,[drama],5.0


In [11]:
print(data_1.iloc[:50,0].to_list())

['tt0000009', 'tt0000147', 'tt0000502', 'tt0000574', 'tt0000591', 'tt0000615', 'tt0000630', 'tt0000675', 'tt0000679', 'tt0000838', 'tt0000842', 'tt0000846', 'tt0000850', 'tt0000859', 'tt0000862', 'tt0000867', 'tt0000868', 'tt0000869', 'tt0000879', 'tt0000886', 'tt0000941', 'tt0000947', 'tt0000959', 'tt0000984', 'tt0001007', 'tt0001010', 'tt0001028', 'tt0001038', 'tt0001043', 'tt0001049', 'tt0001051', 'tt0001059', 'tt0001069', 'tt0001080', 'tt0001081', 'tt0001082', 'tt0001101', 'tt0001109', 'tt0001112', 'tt0001113', 'tt0001115', 'tt0001116', 'tt0001122', 'tt0001141', 'tt0001142', 'tt0001147', 'tt0001159', 'tt0001163', 'tt0001175', 'tt0001179']


In [7]:
from neo4j import GraphDatabase


url = "neo4j+s://9d0eb28d.databases.neo4j.io"
username = "neo4j"
password = ""

driver = GraphDatabase.driver(url, auth=(username, password))
#with driver.session() as session:
#    session.run("MATCH (n) DETACH DELETE n")
with driver.session() as session:
    result = session.run("MATCH (n) RETURN count(n)")
    count = result.single()[0]
    print(f"La conexión con Aura Neo4j se ha establecido correctamente. Hay {count} nodos en la base de datos.")
    
    

La conexión con Aura Neo4j se ha establecido correctamente. Hay 0 nodos en la base de datos.


In [9]:
for i in unique_genres_list: 
     with driver.session() as session:
        session.execute_write(
            lambda tx: tx.run("MERGE (m:Genre {nombre: $nombre})", nombre=i)
        )
for index, row in data_1.iloc[:50,:].iterrows():
    movie_id = row["tconst"]
    title = row["originalTitle"]
    genres = row["genres"] 
    with driver.session() as session:
        session.execute_write(
            lambda tx: tx.run("MERGE (m:Movie {nombre: $nombre,id: $id})", nombre=title, id = movie_id)
        )

    

In [11]:
for index, row in data_1.iloc[:50,:].iterrows():
    movie_id = row["tconst"]
    title = row["originalTitle"]
    genres = row["genres"]        
    with driver.session() as session:
        for genre in genres:
            session.execute_write(
                lambda tx: tx.run("MATCH (g:Genre {nombre: $nombre}) "
            "MATCH (m:Movie {id: $id}) "
            "CREATE (g)-[:IN_GENRE]->(m)"
            , nombre=genre, id=movie_id)
            )